# DisALEXI Single Image Example

In [ ]:
import ee
from IPython.display import Image

from disalexi import disalexi
from disalexi import landsat
disalexi = reload(disalexi)
landsat = reload(landsat)

ee.Initialize()

ndvi_palette = ['#EFE7E1', '#003300']
ta_palette = ['#ff0000', '#ffff00', '#00ffff', '#0000ff']
et_palette = [
  'DEC29B', 'E6CDA1', 'EDD9A6', 'F5E4A9', 'FFF4AD', 'C3E683', '6BCC5C', 
  '3BB369', '20998F', '1C8691', '16678A', '114982', '0B2C7A']

#### Prep the image

In [ ]:
# scene_id = 'LC08_038038_20150717'
landsat_img = ee.Image('LANDSAT/LC08/C01/T1_RT_TOA/LC08_043033_20150602')
input_img = ee.Image(landsat.Landsat(landsat_img).prep())

#### Study area and region mask
Needed for making Ta thumbnail match Landsat thumbnails

In [ ]:
image_region = landsat_img.geometry().bounds(1, 'EPSG:4326').coordinates().getInfo()
image_crs = landsat_img.select(['B2']).projection().crs().getInfo()
image_transform = landsat_img.select(['B2']).projection().getInfo()['transform']
coarse_transform = image_transform[:]
coarse_transform[0] = 240
coarse_transform[4] = 240
# print(image_region)
# print(image_crs)
# print(image_transform)
# print(coarse_transform)

#### Show NDVI

In [ ]:
Image(url=input_img.select(['ndvi']).getThumbURL({
    'min': -0.1, 'max': 0.9, 'palette': ','.join(ndvi_palette)}))

#### Setup DisALEXI

In [ ]:
disalexi_obj = disalexi.DisALEXI(
    input_img, 
    iterations=10,
    elevation=ee.Image('USGS/SRTMGL1_003').rename(['elevation']),
    landcover=ee.Image(
        ee.ImageCollection('users/cgmorton/GlobeLand30_2010').mosaic()) \
            .divide(10).floor().multiply(10).rename(['landcover']),
    lc_type='GLOBELAND30')

#### Compute ALEXI scale air temperature (aggregated LST for now)

In [ ]:
# Projecting the Ta image to the Landsat image for display purposes
ta_img = disalexi_obj.compute_ta() \
    .reproject(crs=image_crs, crsTransform=coarse_transform)
Image(url=ta_img.getThumbURL({
    'min': 280, 'max': 320, 'region': image_region, 'palette': ','.join(ta_palette)}))

#### Compute Landsat scale ET (1.25 * NDVI for now)

In [ ]:
# Skip Ta smoothing for now
et_img = disalexi_obj.compute_et(ta_img)
Image(url=et_img.getThumbURL({
    'min': 0, 'max': 1.25, 'palette': ','.join(et_palette)}))